In [33]:
import pandas as pd
traum=pd.read_csv('Downloads/full_data.csv', sep=';', dtype= {"EAN": "str"}) 

In [35]:
traum.head(5)

,SKU,EAN,MPN,Hersteller,Produktname,Preis,Mindestpreis,Lieferzeit,Farbe,Größe,Versandkosten,Direktverkauf: Freigabe,Lagerbestand,Direktverkauf: Versandart,Direktverkauf: Lieferkosten bis zum Aufstellort,Direktverkauf: Kosten Altgerätemitnahme,Direktverkauf: Preisspanne
0,W-01-30671,4048889008027,190470,Tecline,"Mauspad, blau",11.35,8.69,1 Tag,NaN,NaN,0,1,21,Paketdienst,NaN,NaN,NaN
1,W-01-30688,4048889015254,701090,Tecline,"9pol D-Sub Buchse, Lötversion",9.18,7.03,1 Tag,NaN,NaN,0,1,311,Paketdienst,NaN,NaN,NaN
2,W-01-30696,4048889008621,701250,Tecline,"25pol D-Sub Buchse, Lötversion",9.23,7.07,1 Tag,NaN,NaN,0,1,19,Paketdienst,NaN,NaN,NaN
3,W-80-11141271,4048889008652,10115B,EXERTIS CONNECT,"15pol HD D-Sub Buchse, L?tversion f?r z.B VGA",16.24,12.44,1 Tag,NaN,NaN,0,1,3,Paketdienst,NaN,NaN,NaN
4,W-01-30730,4048889015834,915.170,Bachmann,"Bachmann Kaltgerätekupplung Nr. 794, 70 Grad ...",11.58,8.86,1 Tag,NaN,NaN,0,1,22,Paketdienst,NaN,NaN,NaN


In [36]:
limango.dtypes

Artikelnummer                 object
EAN                           object
PZN                          float64
Herstellerartikelnummer      float64
Hersteller                    object
Produktname                   object
Preis                        float64
Streichpreis                 float64
Lieferzeit                    object
Kategorie                     object
Beschreibung                  object
Bild_URL_main                 object
Bild_URL_extra1               object
Farbe                         object
Größen                        object
Versandkosten Vorkasse       float64
Versandkosten Nachnahme      float64
Versandkosten Kreditkarte    float64
Versandkosten PayPal         float64
Merchant Name                 object
Subcategory                   object
Grundpreis                    object
Deeplink                      object
dtype: object

In [37]:
traum = traum.dropna(subset=['EAN'])

In [38]:
new=traum[['EAN']]
new.head(4)

,EAN
0,4048889008027
1,4048889015254
2,4048889008621
3,4048889008652


In [ ]:
#EAN-Pries-Leadouts(on a monthly basis or daily or last 30 days)
#EAN- product_id-Preis-leadouts

In [17]:
new.to_csv('out.csv', index=False)  

In [39]:
import prestodb
conn=prestodb.dbapi.connect(
    host='presto.apps.eu.idealo.com',
    port=443, user='aishwarya.kumar',
    catalog='hive',
    http_scheme='https',
    schema='default',)
# create the cursor
cur = conn.cursor()
# cursor objects functions and attributes
[thing for thing in dir(cur) if '_' not in thing]

['arraysize',
 'cancel',
 'close',
 'connection',
 'description',
 'execute',
 'executemany',
 'fetchall',
 'fetchmany',
 'fetchone',
 'genall',
 'rowcount',
 'setinputsizes',
 'setoutputsize',
 'stats']

In [40]:
def mapstring(n):
    return "'" + n + "'"

ean_list = list(map(mapstring, traum['EAN'].tolist()))

ean_list

["'4048889008027'",
 "'4048889015254'",
 "'4048889008621'",
 "'4048889008652'",
 "'4048889015834'",
 "'4048889015841'",
 "'4048889009024'",
 "'4017538710406'",
 "'4048889004852'",
 "'4048889004937'",
 "'4048889004944'",
 "'4048889004760'",
 "'4048889004838'",
 "'4048889007099'",
 "'4048889007440'",
 "'4016514014712'",
 "'4016514014729'",
 "'4710423770751'",
 "'4012740837613'",
 "'4048889009475'",
 "'4048889005620'",
 "'4048889005637'",
 "'4048889005651'",
 "'4048889015056'",
 "'4048889015094'",
 "'4012740138192'",
 "'4048889011249'",
 "'4048889011256'",
 "'4048889011270'",
 "'4048889011300'",
 "'4048889011317'",
 "'4048889011331'",
 "'4048889011362'",
 "'4048889011379'",
 "'4048889011393'",
 "'4048889011423'",
 "'4048889011485'",
 "'4048889006610'",
 "'4048889002131'",
 "'4048889006306'",
 "'4049759039868'",
 "'4048889015605'",
 "'4048889012710'",
 "'4048889012772'",
 "'4048889012895'",
 "'4048889012956'",
 "'4048889013014'",
 "'4048889013137'",
 "'4048889013250'",
 "'4048889005644'",


In [20]:
len(ean_list)
ean_list=ean_list[:1000]

In [41]:
# Yield successive n-sized
# chunks from l.
def divide_chunks(l, n):
      
    # looping till length l
    for i in range(0, len(l), n): 
        yield l[i:i + n]
  
# How many elements each
# list should have
n = 50000
  
ean_list_chunks = list(divide_chunks(ean_list, n))
len(ean_list_chunks)

4

In [43]:
from IPython.display import display

def maptoquery(n):
    sql='''
WITH eans AS (
        SELECT DISTINCT ean,id as product_id
        FROM dl_inventory_analytics_prod.current_product_offers
        CROSS JOIN UNNEST(eans) as t(ean)
        WHERE ean in (''' + ','.join(n) + ''')
  )
  
  select distinct ean,product_id,root_category_name
  from dl_inventory_intelligence_prod.tidy_category_tree c join eans e on 
  c.category_id=e.category_id
      
  '''
    print("Execute Query.")
    cur.execute(sql) # this was a multiline string.
    records = cur.fetchall()
    print("Total rows are: ", len(records))
    
    colnames = [col[0] for col in cur.description]
    colnames[0:6]
    return pd.DataFrame(records, columns=colnames)
    
ean_list_result = list(map(maptoquery, ean_list_chunks))


Execute Query.
Total rows are:  28776
Execute Query.
Total rows are:  21446
Execute Query.
Total rows are:  5813
Execute Query.
Total rows are:  1878


In [47]:
df_result = pd.concat(ean_list_result)

In [48]:
n = df_result.nunique(axis=0)
n

ean                   57913
product_id            55080
root_category_name       10
dtype: int64

In [44]:
n = traum.nunique(axis=0)
n

SKU                                                152803
EAN                                                152803
MPN                                                 99520
Hersteller                                           2495
Produktname                                        124867
Preis                                               29550
Mindestpreis                                        27194
Lieferzeit                                              4
Farbe                                                   0
Größe                                                   0
Versandkosten                                           1
Direktverkauf: Freigabe                                 1
Lagerbestand                                         1561
Direktverkauf: Versandart                               2
Direktverkauf: Lieferkosten bis zum Aufstellort         0
Direktverkauf: Kosten Altgerätemitnahme                 0
Direktverkauf: Preisspanne                              0
dtype: int64

In [24]:
df_result.head(10)

,EAN,offer_id,product_id,offer_pos,type,min_price,count
0,4005556214228,3394967126681849913,327084.0,-1.0,checkout,16.49,1.0
1,4005556214228,3394967126681849913,327084.0,2.0,offer,16.49,1.0
2,4005556214228,9706766438,327084.0,4.0,offer,16.49,1.0
3,5012909005272,3420428631506157799,570050.0,1.0,checkout,49.26,9.0
4,5012909005272,3402871041104550294,570050.0,1.0,offer,49.79,3.0
5,5012909005272,3363534124085155438,570050.0,1.0,offer,None,2.0
6,5012909005272,3420428631506157799,570050.0,2.0,checkout,49.26,2.0
7,5012909005272,3404425334852948887,570050.0,2.0,offer,45.99,2.0
8,5012909005272,3420428631506157799,570050.0,2.0,offer,49.26,3.0
9,5012909005272,3404425334852948887,570050.0,3.0,offer,45.99,2.0


In [115]:
limango_2=limango[['EAN','Preis']]
limango_2=limango_2.rename(columns={"Preis": "min_price"})
limango_2=limango_2.dropna()
limango_2=limango_2.astype({'EAN': 'int64'})
df_result_2=df_result.dropna()
result = pd.merge(df_result_2, limango_2, on=["EAN", "min_price"], how='inner')

In [151]:
#result=result[((result['EAN']==4033157410367) & (result['min_price']==259.0))]

In [153]:
result_2=result.groupby(['EAN','product_id','min_price']).agg({'count': ['sum']})
result_2

count
                                     sum
EAN           product_id min_price      
4033157410367 669476.0   259.0       2.0
              3034858.0  259.0       6.0

In [157]:
result.count()

EAN           4314
offer_id      4314
product_id    4314
offer_pos     4314
type          4314
min_price     4314
count         4314
dtype: int64

In [160]:
df_result.count()

EAN           4486574
offer_id      4486574
product_id     231413
offer_pos      231498
type           231498
min_price     2929780
count          231498
dtype: int64

In [162]:
n = df_result.nunique(axis=0)
n

EAN            180537
offer_id      4048194
product_id      37048
offer_pos         272
type                3
min_price         933
count             195
dtype: int64

In [169]:
result.to_csv('limago_LOCO_result.csv', index=False)  

In [170]:
result.count()

EAN           4314
offer_id      4314
product_id    4314
offer_pos     4314
type          4314
min_price     4314
count         4314
dtype: int64

In [172]:
a=result.groupby(['EAN','product_id','min_price']).agg({'count': ['sum']})

In [173]:
a

,,,count
,,,sum
EAN,product_id,min_price,
191190219553,6613774.0,80.0,1.0
191190219577,6613774.0,80.0,1.0
192824082994,5532676.0,85.0,1.0
193145645523,200555503.0,59.0,3.0
193655216572,7029478.0,39.0,3.0
...,...,...,...
9010159299485,200745481.0,120.0,1.0
9010159884131,200135860.0,99.0,1.0


In [175]:
a.to_csv('output.csv', index=True)

In [28]:
df_result_2.head(10)

NameError: name 'df_result_2' is not defined